# Deep Learning

## Assignment 3

Previously in 2_fullyconnected.ipynb, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in 1_notmnist.ipynb.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---

## Problem 1

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.


---

Logistic :

In [5]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape = (batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,
                                    shape = (batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Parameters
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #Computation
    logits = tf.matmul(tf_train_dataset,weights)+biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels,logits= logits) + beta*tf.nn.l2_loss(weights))
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [17]:
num_steps = 3001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_label = train_labels[offset:(offset+batch_size)]
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_label}
        _,l,prediction = session.run([optimizer,loss,train_prediction],feed_dict = feed_dict)
        if (step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(prediction, batch_label))
            print("Validation accuracy: %.1f%%" % accuracy(
                    valid_prediction.eval(),valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.087166
Minibatch accuracy: 11.7%
Validation accuracy: 15.9%
Minibatch loss at step 500: 2.554112
Minibatch accuracy: 79.7%
Validation accuracy: 76.2%
Minibatch loss at step 1000: 1.725648
Minibatch accuracy: 78.1%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 0.976111
Minibatch accuracy: 82.0%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 0.866525
Minibatch accuracy: 89.1%
Validation accuracy: 80.7%
Minibatch loss at step 2500: 0.864004
Minibatch accuracy: 78.1%
Validation accuracy: 81.6%
Minibatch loss at step 3000: 0.755946
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Test accuracy: 88.9%


In [23]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    
    #constants
    tf_train_dataset = tf.placeholder(tf.float32,shape=[batch_size,image_size*image_size])
    tf_train_labels = tf.placeholder(tf.float32,shape=[batch_size,num_labels])
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    
    #variables
    weight_1 = tf.Variable(tf.truncated_normal([image_size*image_size,1024]))
    biases_1 = tf.Variable(tf.zeros([1024]))
    
    weight_2 = tf.Variable(tf.truncated_normal([1024,num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    #Computation
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset,weight_1)+biases_1)
    
    logits = tf.matmul(hidden,weight_2)+biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels,logits=logits)+beta*tf.nn.l2_loss(weight_2) + beta*tf.nn.l2_loss(weight_1))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #prediction
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weight_1)+biases_1),weight_2)+biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weight_1)+biases_1),weight_2)+biases_2)

In [24]:
num_steps = 3001

with tf.Session(graph = graph) as sess:
    print("initialized")
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        offset = batch_size*step%(train_dataset.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        _,l,predictions = sess.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if step%500==0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Train accuracy %.1f%%" % accuracy(predictions,batch_labels))
            print("Valid accuracy %.1f%%" % accuracy(valid_prediction.eval(),valid_labels))
    print("Test accuracy %.1f%%" % accuracy(test_prediction.eval(),test_labels))

initialized
Minibatch loss at step 0: 645.293640
Train accuracy 14.1%
Valid accuracy 33.1%
Minibatch loss at step 500: 199.374329
Train accuracy 81.2%
Valid accuracy 78.9%
Minibatch loss at step 1000: 115.399429
Train accuracy 81.2%
Valid accuracy 81.4%
Minibatch loss at step 1500: 68.824875
Train accuracy 91.4%
Valid accuracy 83.6%
Minibatch loss at step 2000: 41.233017
Train accuracy 92.2%
Valid accuracy 84.7%
Minibatch loss at step 2500: 25.168875
Train accuracy 89.8%
Valid accuracy 85.5%
Minibatch loss at step 3000: 15.483320
Train accuracy 84.4%
Valid accuracy 86.4%
Test accuracy 93.1%


---

## Problem 2

Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [26]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    
    #constants
    tf_train_dataset = tf.placeholder(tf.float32,shape=[batch_size,image_size*image_size])
    tf_train_labels = tf.placeholder(tf.float32,shape=[batch_size,num_labels])
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    
    #variables
    weight_1 = tf.Variable(tf.truncated_normal([image_size*image_size,1024]))
    biases_1 = tf.Variable(tf.zeros([1024]))
    
    weight_2 = tf.Variable(tf.truncated_normal([1024,num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    #Computation
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset,weight_1)+biases_1)
    
    logits = tf.matmul(hidden,weight_2)+biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels,logits=logits)+beta*tf.nn.l2_loss(weight_2) + beta*tf.nn.l2_loss(weight_1))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #prediction
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weight_1)+biases_1),weight_2)+biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weight_1)+biases_1),weight_2)+biases_2)

In [27]:
num_steps = 3001

sam_train_data = train_dataset[:500,:]
sam_train_labels = train_labels[:500,:]

with tf.Session(graph = graph) as sess:
    print("initialized")
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        offset = batch_size*step%(sam_train_data.shape[0]-batch_size)
        batch_data = sam_train_data[offset:(offset+batch_size),:]
        batch_labels = sam_train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        _,l,predictions = sess.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if step%500==0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Train accuracy %.1f%%" % accuracy(predictions,batch_labels))
            print("Valid accuracy %.1f%%" % accuracy(valid_prediction.eval(),valid_labels))
    print("Test accuracy %.1f%%" % accuracy(test_prediction.eval(),test_labels))

initialized
Minibatch loss at step 0: 633.627808
Train accuracy 15.6%
Valid accuracy 27.8%
Minibatch loss at step 500: 190.189850
Train accuracy 100.0%
Valid accuracy 75.4%
Minibatch loss at step 1000: 115.341507
Train accuracy 100.0%
Valid accuracy 75.4%
Minibatch loss at step 1500: 69.949417
Train accuracy 100.0%
Valid accuracy 75.4%
Minibatch loss at step 2000: 42.421204
Train accuracy 100.0%
Valid accuracy 75.2%
Minibatch loss at step 2500: 25.726685
Train accuracy 100.0%
Valid accuracy 75.2%
Minibatch loss at step 3000: 15.602524
Train accuracy 100.0%
Valid accuracy 75.4%
Test accuracy 82.3%


---

## Problem 3

Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.
What happens to our extreme overfitting case?

---

In [116]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    
    #constants
    tf_train_dataset = tf.placeholder(tf.float32,shape=[batch_size,image_size*image_size])
    tf_train_labels = tf.placeholder(tf.float32,shape=[batch_size,num_labels])
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_keep_prob = tf.constant(0.5)
    #variables
    weight_1 = tf.Variable(tf.truncated_normal([image_size*image_size,1024]))
    biases_1 = tf.Variable(tf.zeros([1024]))
    
    weight_2 = tf.Variable(tf.truncated_normal([1024,num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    #Computation
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset,weight_1)+biases_1)
    drop_hidden = tf.nn.dropout(hidden,keep_prob=tf_keep_prob)
    
    logits = tf.matmul(drop_hidden,weight_2)+biases_2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels,logits=logits)+beta*tf.nn.l2_loss(weight_2) + beta*tf.nn.l2_loss(weight_1))
    
    #optimizer
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.001, global_step, 3000,0.96,staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    #optimizer = tf.train.GradientDescentOptimizer(0.03).minimize(loss)
    
    
    #prediction
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weight_1)+biases_1),weight_2)+biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weight_1)+biases_1),weight_2)+biases_2)

In [8]:
num_steps = 3001

with tf.Session(graph = graph) as sess:
    print("initialized")
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        offset = batch_size*step%(train_dataset.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        _,l,predictions = sess.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if step%500==0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Train accuracy %.1f%%" % accuracy(predictions,batch_labels))
            print("Valid accuracy %.1f%%" % accuracy(valid_prediction.eval(),valid_labels))
        #global_step+=1
    print("Test accuracy %.1f%%" % accuracy(test_prediction.eval(),test_labels))

initialized
Minibatch loss at step 0: 4144.485352
Train accuracy 10.2%
Valid accuracy 11.7%
Minibatch loss at step 500: 714.283081
Train accuracy 69.5%
Valid accuracy 71.1%
Minibatch loss at step 1000: 723.118103
Train accuracy 74.2%
Valid accuracy 73.6%
Minibatch loss at step 1500: 570.418579
Train accuracy 80.5%
Valid accuracy 74.8%
Minibatch loss at step 2000: 602.315491
Train accuracy 77.3%
Valid accuracy 75.7%
Minibatch loss at step 2500: 572.526733
Train accuracy 75.8%
Valid accuracy 76.0%
Minibatch loss at step 3000: 561.943848
Train accuracy 72.7%
Valid accuracy 76.5%
Test accuracy 84.3%


Limited Dataset Solution :

In [36]:
num_steps = 3001

sam_train_data = train_dataset[:500,:]
sam_train_labels = train_labels[:500,:]

with tf.Session(graph = graph) as sess:
    print("initialized")
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        offset = batch_size*step%(sam_train_data.shape[0]-batch_size)
        batch_data = sam_train_data[offset:(offset+batch_size),:]
        batch_labels = sam_train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        _,l,predictions = sess.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if step%500==0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Train accuracy %.1f%%" % accuracy(predictions,batch_labels))
            print("Valid accuracy %.1f%%" % accuracy(valid_prediction.eval(),valid_labels))
    print("Test accuracy %.1f%%" % accuracy(test_prediction.eval(),test_labels))

initialized
Minibatch loss at step 0: 768.750000
Train accuracy 9.4%
Valid accuracy 32.3%
Minibatch loss at step 500: 191.571442
Train accuracy 100.0%
Valid accuracy 78.4%
Minibatch loss at step 1000: 116.235710
Train accuracy 100.0%
Valid accuracy 78.5%
Minibatch loss at step 1500: 70.520401
Train accuracy 100.0%
Valid accuracy 78.2%
Minibatch loss at step 2000: 42.773659
Train accuracy 100.0%
Valid accuracy 78.7%
Minibatch loss at step 2500: 25.942890
Train accuracy 100.0%
Valid accuracy 78.4%
Minibatch loss at step 3000: 15.733993
Train accuracy 100.0%
Valid accuracy 78.5%
Test accuracy 85.8%


---

## Problem 4

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.
One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)


---

In [15]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    
    #constants
    tf_train_dataset = tf.placeholder(tf.float32,shape=[batch_size,image_size*image_size])
    tf_train_labels = tf.placeholder(tf.float32,shape=[batch_size,num_labels])
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_keep_prob = tf.constant(1.0)
    #variables
    weight_1 = tf.Variable(tf.truncated_normal([image_size*image_size,1024]))
    biases_1 = tf.Variable(tf.zeros([1024]))
    
    weight_2 = tf.Variable(tf.truncated_normal([1024,256]))
    biases_2 = tf.Variable(tf.zeros([256]))
    
    #weight_3 = tf.Variable(tf.truncated_normal([256,128]))
    #biases_3 = tf.Variable(tf.zeros([128]))
    
    weight_final = tf.Variable(tf.truncated_normal([256,num_labels]))
    biases_final = tf.Variable(tf.zeros([num_labels]))
    
    #Computation
    
    hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset,weight_1)+biases_1)
    #drop_hidden_1 = tf.nn.dropout(hidden_1,keep_prob=tf_keep_prob)

    hidden_2 = tf.nn.relu(tf.matmul(hidden_1,weight_2)+biases_2)
    #drop_hidden_2 = tf.nn.dropout(hidden_2,keep_prob=tf_keep_prob)

    #hidden_3 = tf.nn.relu(tf.matmul(drop_hidden_2,weight_3)+biases_3)
    #drop_hidden_3 = tf.nn.dropout(hidden_3,keep_prob=tf_keep_prob)
    
    logits = tf.matmul(hidden_2,weight_final)+biases_final
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels,logits=logits)+
                          beta*tf.nn.l2_loss(weight_final) +
                         beta*tf.nn.l2_loss(weight_1)  +
                         beta*tf.nn.l2_loss(weight_2) 
                         #beta*tf.nn.l2_loss(weight_3)
                         )
    
    #optimizer
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.001, global_step, 1000,0.90,staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    #prediction
    train_prediction = tf.nn.softmax(logits)
    
    #Validation
    hidden_val_1 = tf.nn.relu(tf.matmul(tf_valid_dataset,weight_1)+biases_1)
    hidden_val_2 = tf.nn.relu(tf.matmul(hidden_val_1,weight_2)+biases_2)
    #hidden_val_3 = tf.nn.relu(tf.matmul(hidden_val_2,weight_3)+biases_3)
    valid_prediction = tf.nn.softmax(tf.matmul(hidden_val_2,weight_final)+biases_final)
    
    #Test
    hidden_test_1 = tf.nn.relu(tf.matmul(tf_test_dataset,weight_1)+biases_1)
    hidden_test_2 = tf.nn.relu(tf.matmul(hidden_test_1,weight_2)+biases_2)
    #hidden_test_3 = tf.nn.relu(tf.matmul(hidden_test_2,weight_3)+biases_3)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_test_2,weight_final)+biases_final)

In [16]:
num_steps = 9001

with tf.Session(graph = graph) as sess:
    print("initialized")
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        offset = batch_size*step%(train_dataset.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        _,l,predictions = sess.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if step%500==0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Train accuracy %.1f%%" % accuracy(predictions,batch_labels))
            print("Valid accuracy %.1f%%" % accuracy(valid_prediction.eval(),valid_labels))
        #global_step+=1
    print("Test accuracy %.1f%%" % accuracy(test_prediction.eval(),test_labels))

initialized
Minibatch loss at step 0: 3315.189209
Train accuracy 14.1%
Valid accuracy 14.1%
Minibatch loss at step 500: 639.918518
Train accuracy 80.5%
Valid accuracy 75.0%
Minibatch loss at step 1000: 669.706482
Train accuracy 75.0%
Valid accuracy 76.0%
Minibatch loss at step 1500: 536.940186
Train accuracy 80.5%
Valid accuracy 77.0%
Minibatch loss at step 2000: 541.268677
Train accuracy 78.9%
Valid accuracy 77.5%
Minibatch loss at step 2500: 502.972504
Train accuracy 79.7%
Valid accuracy 77.5%
Minibatch loss at step 3000: 481.505310
Train accuracy 75.8%
Valid accuracy 77.4%
Minibatch loss at step 3500: 485.201111
Train accuracy 77.3%
Valid accuracy 77.5%
Minibatch loss at step 4000: 456.499939
Train accuracy 78.9%
Valid accuracy 77.1%
Minibatch loss at step 4500: 447.071747
Train accuracy 79.7%
Valid accuracy 77.5%
Minibatch loss at step 5000: 447.720215
Train accuracy 82.0%
Valid accuracy 77.5%
Minibatch loss at step 5500: 446.184814
Train accuracy 74.2%
Valid accuracy 77.0%
Minibat